# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

## Ensure that the dependencies and the Geoapify API key is imported correctly

In [26]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key
from config import hotel_key

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [27]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")

# Display sample data
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Bluff,NZ,-46.6000,168.3333,63.37,72,84,8.59,broken clouds
1,1,Meulaboh,ID,4.1363,96.1285,74.10,93,100,5.48,overcast clouds
2,2,Atar,MR,20.5169,-13.0499,76.57,15,0,6.17,clear sky
3,3,College,US,64.8569,-147.8028,12.20,83,100,10.36,light snow
4,4,Ancud,CL,-41.8697,-73.8203,55.83,85,93,2.46,overcast clouds
5,5,Thompson,CA,55.7435,-97.8558,26.76,93,100,3.44,light shower snow
6,6,Chuy,UY,-33.6971,-53.4616,63.36,60,72,10.76,broken clouds
7,7,Dicabisagan,PH,17.0818,122.4157,77.23,89,100,8.61,light rain
8,8,Gold Coast,AU,-28.0000,153.4333,83.16,69,100,18.41,overcast clouds
9,9,Kapaa,US,22.0752,-159.3190,84.18,71,66,7.00,broken clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

In [28]:
# Prompt the user to enter minimum and maximum temperature criteria.
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

What is the minimum temperature you would like for your vacation? 70
What is the maximum temperature you would like for your vacation? 90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [29]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
preferred_cities_df = city_data_df.loc[
    (city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)
]

# Display sample data
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Meulaboh,ID,4.1363,96.1285,74.10,93,100,5.48,overcast clouds
2,2,Atar,MR,20.5169,-13.0499,76.57,15,0,6.17,clear sky
7,7,Dicabisagan,PH,17.0818,122.4157,77.23,89,100,8.61,light rain
8,8,Gold Coast,AU,-28.0000,153.4333,83.16,69,100,18.41,overcast clouds
9,9,Kapaa,US,22.0752,-159.3190,84.18,71,66,7.00,broken clouds
11,11,Sotouboua,TG,8.5667,0.9833,75.61,22,5,3.09,clear sky
12,12,Hambantota,LK,6.1241,81.1185,73.47,80,90,5.17,overcast clouds
14,14,Geraldton,AU,-28.7667,114.6000,82.81,21,0,16.11,clear sky
16,16,Rikitea,PF,-23.1203,-134.9692,76.10,78,2,21.12,clear sky
17,17,Saint-Philippe,RE,-21.3585,55.7679,72.99,85,32,9.51,light rain


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [30]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities_df = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Meulaboh,ID,4.1363,96.1285,74.10,93,100,5.48,overcast clouds
2,2,Atar,MR,20.5169,-13.0499,76.57,15,0,6.17,clear sky
7,7,Dicabisagan,PH,17.0818,122.4157,77.23,89,100,8.61,light rain
8,8,Gold Coast,AU,-28.0000,153.4333,83.16,69,100,18.41,overcast clouds
9,9,Kapaa,US,22.0752,-159.3190,84.18,71,66,7.00,broken clouds
11,11,Sotouboua,TG,8.5667,0.9833,75.61,22,5,3.09,clear sky
12,12,Hambantota,LK,6.1241,81.1185,73.47,80,90,5.17,overcast clouds
14,14,Geraldton,AU,-28.7667,114.6000,82.81,21,0,16.11,clear sky
16,16,Rikitea,PF,-23.1203,-134.9692,76.10,78,2,21.12,clear sky
17,17,Saint-Philippe,RE,-21.3585,55.7679,72.99,85,32,9.51,light rain


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [31]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
new_hotel_df = clean_travel_cities_df.copy(deep=True)
hotel_df = new_hotel_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]]

## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [32]:
# Create a new column "Hotel Name".
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Meulaboh,ID,74.10,overcast clouds,4.1363,96.1285,
2,Atar,MR,76.57,clear sky,20.5169,-13.0499,
7,Dicabisagan,PH,77.23,light rain,17.0818,122.4157,
8,Gold Coast,AU,83.16,overcast clouds,-28.0000,153.4333,
9,Kapaa,US,84.18,broken clouds,22.0752,-159.3190,
11,Sotouboua,TG,75.61,clear sky,8.5667,0.9833,
12,Hambantota,LK,73.47,overcast clouds,6.1241,81.1185,
14,Geraldton,AU,82.81,clear sky,-28.7667,114.6000,
16,Rikitea,PF,76.10,clear sky,-23.1203,-134.9692,
17,Saint-Philippe,RE,72.99,light rain,-21.3585,55.7679,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [33]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": hotel_key,
    "limit":20
}

## Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found"

In [34]:
# Iterate through the hotel DataFrame

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places"

    # Make request and retrieve the JSON data by using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found set the hotel name as "No hotel found".
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Meulaboh - nearest hotel: Hotel Meuligou
Atar - nearest hotel: فندق سكليل
Dicabisagan - nearest hotel: No hotel found
Gold Coast - nearest hotel: Zenith
Kapaa - nearest hotel: Pono Kai Resort
Sotouboua - nearest hotel: Verger
Hambantota - nearest hotel: Oasis Aurweda Resort Hambathota
Geraldton - nearest hotel: Ocean Centre Hotel
Rikitea - nearest hotel: Chez Bianca & Benoit
Saint-Philippe - nearest hotel: Le Baril
Lekoni - nearest hotel: Le rdv des chasseurs
Nelson Bay - nearest hotel: No hotel found
Jardim - nearest hotel: Hotel Uchôa
Manicore - nearest hotel: No hotel found
Sao Joao Da Barra - nearest hotel: Pousada Mediterrâneo
Mataura - nearest hotel: No hotel found
Grand Gaube - nearest hotel: Coco Villa
Kuala Terengganu - nearest hotel: Uncle Homestay
Puerto Ayora - nearest hotel: Hostal La Mirada De Solitario George
Te Anau - nearest hotel: Lakeside Motels & Apartments
Bengkulu - nearest hotel: Dena Hotel
Hithadhoo - nearest hotel: Pebbles Inn
Mahebourg - 

Mareeba - nearest hotel: The Gateway Hotel
Paranapanema - nearest hotel: No hotel found
Hervey Bay - nearest hotel: Hervey Bay Hotel
Bridgetown - nearest hotel: Radisson Aquatica Resort
Poum - nearest hotel: No hotel found
Eseka - nearest hotel: No hotel found
Sao Paulo Do Potengi - nearest hotel: No hotel found
Rio Cuarto - nearest hotel: Marluc
Bubaque - nearest hotel: Chez Julio
Maun - nearest hotel: Center Lodge Conference Center
Hukuntsi - nearest hotel: Entabeni Hotel
Sambava - nearest hotel: Royal Sava Hôtel
Carmen - nearest hotel: Holiday Inn Express
Champerico - nearest hotel: Hotel y Restaurante El Submarino
Oussouye - nearest hotel: Campement Alulum
Gewane - nearest hotel: No hotel found
Kroya - nearest hotel: Setia Hotel
Pauini - nearest hotel: No hotel found
Raga - nearest hotel: No hotel found
Tabou - nearest hotel: hôtel le rochet
Kupang - nearest hotel: BIB & LANG GUEST HOUSE
Lazaro Cardenas - nearest hotel: Hotel Sol del Pacífico
Kahului - nearest hotel: Maui Seaside H

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Meulaboh,ID,74.10,overcast clouds,4.1363,96.1285,Hotel Meuligou
2,Atar,MR,76.57,clear sky,20.5169,-13.0499,فندق سكليل
7,Dicabisagan,PH,77.23,light rain,17.0818,122.4157,No hotel found
8,Gold Coast,AU,83.16,overcast clouds,-28.0000,153.4333,Zenith
9,Kapaa,US,84.18,broken clouds,22.0752,-159.3190,Pono Kai Resort
...,...,...,...,...,...,...,...
685,Gigmoto,PH,81.25,overcast clouds,13.7796,124.3900,No hotel found
689,San-Pedro,CI,78.31,clear sky,4.7485,-6.6363,Hôtel La Cité De San Pédro
690,Mandera,KE,80.31,clear sky,3.9366,41.8670,Kornesh Hotel and Lodging
695,Kerema,PG,82.60,overcast clouds,-7.9631,145.7785,Kerema Hotel


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [47]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != "No hotel found"]

# Display sample data
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Meulaboh,ID,74.10,overcast clouds,4.1363,96.1285,Hotel Meuligou
2,Atar,MR,76.57,clear sky,20.5169,-13.0499,فندق سكليل
8,Gold Coast,AU,83.16,overcast clouds,-28.0000,153.4333,Zenith
9,Kapaa,US,84.18,broken clouds,22.0752,-159.3190,Pono Kai Resort
11,Sotouboua,TG,75.61,clear sky,8.5667,0.9833,Verger
12,Hambantota,LK,73.47,overcast clouds,6.1241,81.1185,Oasis Aurweda Resort Hambathota
14,Geraldton,AU,82.81,clear sky,-28.7667,114.6000,Ocean Centre Hotel
16,Rikitea,PF,76.10,clear sky,-23.1203,-134.9692,Chez Bianca & Benoit
17,Saint-Philippe,RE,72.99,light rain,-21.3585,55.7679,Le Baril
18,Lekoni,GA,70.11,overcast clouds,-1.5843,14.2591,Le rdv des chasseurs


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [51]:
# Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [54]:
# Configure the map plot
clean_hotel_map = clean_hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Max Temp",
    scale = .5,
    hover_cols = ["City", "Country", "Current Description"]
)

# Display the map
clean_hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Max Temp,City,Country,Current Description)